## Stitching Networks between Chameleon and CityLab

An example that can be used to stitch to the Antwerp CityLab

#### Modules 

- [Reserve Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)
- [Get Network by Name](../modules-python/network/get_network_by_name.ipynb)

TODO: add the rest of the module links when they are ready

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [2]:
import json
import os
import chi

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project
chi.set('project_name', 'ExoGENI@Chameleon') # Replace with your project name

#Set the region
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

#Insert keypair name
key_name = 'pruth-jupyter'  # Change to your keypair

#GENI Pem file
geni_pem_file='/home/pruth/work/geni-pruth.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
stitched_network_name = network_name+"Stitched"
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name_network = username+'LeaseNet'
lease_name_servers = username+'LeaseServers'
lease_name_stitch = username+'LeaseStitch'


#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=2

#Network Config
cidr='192.168.42.0/24'

#optionally set the OpenFlow controller (set ot None for non-OpenFlow swtich)
#of_controller_ip=None
of_controller_ip="192.5.87.215"
of_controller_port="6653"


#Set the name of the VFC
vswitch_name="pruthSDN"

#CityLab Stitchport (posible vlan 1065-1069)
stitchport_url='http://geni-orca.renci.org/owl/ion.rdf#AL2S/iMind/Cisco/6509/TengigabitEthernet/1/1/ethernet'
stitchport_vlan=1065

## Create the Network (OpenFlow Optional)

#### Create a Lease at Chicago

In [3]:


# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)


# Build list of reservations (in this case there is only one reservation)
reservation_list = []

if of_controller_ip:
    print("OpenFlow")
    add_network_reservation(reservation_list, 
                            network_name=network_name, 
                            physical_network='physnet1',
                            of_controller_ip=of_controller_ip, 
                            of_controller_port=of_controller_port, 
                            vswitch_name=vswitch_name)
else:
    print("No OpenFlow")
    add_network_reservation(reservation_list, 
                            network_name=network_name, 
                            physical_network='physnet1',
                            vswitch_name=vswitch_name)
                           

# Create the lease
chi.blazar().lease.create(name=lease_name_network, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
network_lease = get_lease_by_name(lease_name_network)
    
#Print the lease info
print(json.dumps(network_lease, indent=2))

OpenFlow
{
  "status": "PENDING",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLeaseNet",
  "end_date": "2020-07-24T17:15:00.000000",
  "reservations": [
    {
      "status": "pending",
      "lease_id": "af1c661e-25e3-4e0e-bcb9-66b3730d4041",
      "resource_id": "eebb0976-77e1-4e16-b949-41c2f2423226",
      "created_at": "2020-07-23 17:15:36",
      "updated_at": "2020-07-23 17:15:38",
      "missing_resources": false,
      "id": "a954cc11-ca69-43d0-862a-ee0c63288e7f",
      "resource_type": "network",
      "resources_changed": false
    }
  ],
  "created_at": "2020-07-23 17:15:36",
  "updated_at": "2020-07-23 17:15:38",
  "events": [
    {
      "status": "UNDONE",
      "lease_id": "af1c661e-25e3-4e0e-bcb9-66b3730d4041",
      "event_type": "before_end_lease",
      "created_at": "2020-07-23 17:15:38",
      "updated_at": null,
      "time": "2020-07-23T17:16:00.000000",
      "id": "077b6045-f7ae-4acb-9378-1da79956378c"
    },
    {
      "status": "UNDONE",

#### Get the Nework Reservation

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [4]:
#Get the lease by name
network_lease = get_lease_by_name(lease_name_network)

network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', network_lease['reservations']))[0]['id']

print("network_reservation_id: " + network_reservation_id)

network_reservation_id: a954cc11-ca69-43d0-862a-ee0c63288e7f


In [5]:
#Get the network
network = get_network_by_name(network_name)
#print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
network_vlan = network['provider:segmentation_id']
print('network_vlan: ' + str(network_vlan))

Network ID: b6657377-18c2-4b7c-98c9-89dda0ec16b0
network_vlan: 3250


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [6]:
subnet = add_subnet(subnet_name, network_name, cidr=cidr)

print(json.dumps(subnet, indent=2))

{
  "subnets": [
    {
      "service_types": [],
      "description": "",
      "enable_dhcp": true,
      "tags": [],
      "network_id": "b6657377-18c2-4b7c-98c9-89dda0ec16b0",
      "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
      "created_at": "2020-07-23T17:16:31Z",
      "dns_nameservers": [],
      "updated_at": "2020-07-23T17:16:31Z",
      "gateway_ip": "192.168.42.1",
      "ipv6_ra_mode": null,
      "allocation_pools": [
        {
          "start": "192.168.42.2",
          "end": "192.168.42.254"
        }
      ],
      "host_routes": [],
      "revision_number": 0,
      "ip_version": 4,
      "ipv6_address_mode": null,
      "cidr": "192.168.42.0/24",
      "project_id": "d9faac3973a847f1b718fa765fe312e2",
      "id": "7c6b4fe8-60f8-414a-9010-a10504461049",
      "subnetpool_id": null,
      "name": "pruthsubnet"
    }
  ]
}


#### Add a Router

TODO: add links here

In [7]:
router = create_router(router_name, gw_network_name='public')

print(json.dumps(router, indent=2))

{
  "router": {
    "status": "ACTIVE",
    "external_gateway_info": {
      "network_id": "44b38c44-2a42-4b6d-b129-6c8f1b2a1375",
      "enable_snat": true,
      "external_fixed_ips": [
        {
          "subnet_id": "c3950603-9e04-4cc5-be8d-1efbfe59fc0a",
          "ip_address": "192.5.87.140"
        }
      ]
    },
    "availability_zone_hints": [],
    "availability_zones": [],
    "description": "",
    "tags": [],
    "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
    "created_at": "2020-07-23T17:16:35Z",
    "admin_state_up": true,
    "updated_at": "2020-07-23T17:16:36Z",
    "flavor_id": null,
    "revision_number": 3,
    "routes": [],
    "project_id": "d9faac3973a847f1b718fa765fe312e2",
    "id": "6f706200-c41b-48c7-8e46-b2647641c641",
    "name": "pruthRouter"
  }
}


#### Attach the Router and Subnet

TODO: Add links here

In [8]:
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

{'network_id': 'b6657377-18c2-4b7c-98c9-89dda0ec16b0',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '7c6b4fe8-60f8-414a-9010-a10504461049',
 'subnet_ids': ['7c6b4fe8-60f8-414a-9010-a10504461049'],
 'port_id': '38c74041-9652-479d-a6c0-d8c4547c8b7e',
 'id': '6f706200-c41b-48c7-8e46-b2647641c641'}

#### Add a Circuit Stitched to ExoGENI

In [9]:
#Set the region
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []

add_network_reservation(reservation_list,
                        network_name=stitched_network_name, 
                        physical_network='exogeni',
                        vswitch_name=vswitch_name)
                           

# Create the lease
chi.blazar().lease.create(name=lease_name_stitch, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
stitched_network_lease = get_lease_by_name(lease_name_stitch)
    
#Print the lease info
print(json.dumps(stitched_network_lease, indent=2))

{
  "status": "PENDING",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLeaseStitch",
  "end_date": "2020-07-24T17:16:00.000000",
  "reservations": [
    {
      "status": "pending",
      "lease_id": "3a089e4b-91fe-43ba-b7e0-77154fa93e91",
      "resource_id": "65789930-45eb-49b2-bd13-1f585fd8c3bd",
      "created_at": "2020-07-23 17:16:45",
      "updated_at": "2020-07-23 17:16:47",
      "missing_resources": false,
      "id": "40d19797-0165-4403-ab37-c6216a90f52a",
      "resource_type": "network",
      "resources_changed": false
    }
  ],
  "created_at": "2020-07-23 17:16:45",
  "updated_at": "2020-07-23 17:16:47",
  "events": [
    {
      "status": "UNDONE",
      "lease_id": "3a089e4b-91fe-43ba-b7e0-77154fa93e91",
      "event_type": "start_lease",
      "created_at": "2020-07-23 17:16:47",
      "updated_at": null,
      "time": "2020-07-23T17:17:00.000000",
      "id": "30fc64e5-377d-4bb9-b81a-dad634d9454f"
    },
    {
      "status": "UNDONE",
      "lea

#### Get the Stitched Network Reservation and VLAN

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [10]:
#Get the lease by name
stitch_lease = get_lease_by_name(lease_name_stitch)

stitch_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', stitch_lease['reservations']))[0]['id']

print("stitch_reservation_id: " + stitch_reservation_id)

chi.set('region_name', 'CHI@UC')  
#Get the network
network = get_network_by_name(stitched_network_name)
#print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
exogeni_vlan = network['provider:segmentation_id']
print('exogeni_vlan: ' + str(exogeni_vlan))



stitch_reservation_id: 40d19797-0165-4403-ab37-c6216a90f52a
Network ID: cedf73cd-2474-4aa3-8f62-398d1c51d08f
exogeni_vlan: 3291


## Create the Servers

In [11]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name_servers, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
server_lease = get_lease_by_name(lease_name_servers)
    
#Print the lease info
print(json.dumps(server_lease, indent=2))

{
  "status": "PENDING",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLeaseServers",
  "end_date": "2020-07-24T17:17:00.000000",
  "reservations": [
    {
      "status": "pending",
      "lease_id": "c774245b-6ab1-4563-ad8e-e1efe9cbfb6c",
      "resource_id": "e1521ffd-3f7a-4e2b-b0c1-a5c8523568af",
      "network_id": "44b38c44-2a42-4b6d-b129-6c8f1b2a1375",
      "created_at": "2020-07-23 17:17:49",
      "updated_at": "2020-07-23 17:17:51",
      "required_floatingips": [],
      "missing_resources": false,
      "amount": 1,
      "id": "da36c34a-c7fe-432b-932f-1cab12e6806f",
      "resource_type": "virtual:floatingip",
      "resources_changed": false
    },
    {
      "status": "pending",
      "before_end": "default",
      "lease_id": "c774245b-6ab1-4563-ad8e-e1efe9cbfb6c",
      "resource_id": "4a750739-dbc3-4db0-8082-1e1a0cf4fea4",
      "max": 2,
      "created_at": "2020-07-23 17:17:41",
      "min": 2,
      "updated_at": "2020-07-23 17:17:49",
      "m

In [12]:
#Get the lease by name
server_lease = get_lease_by_name(lease_name_servers)

compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', server_lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', server_lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

compute_reservation_id: e30bc6b7-d42c-466e-91cb-cbf5c1269404
floatingip_reservation_id: da36c34a-c7fe-432b-932f-1cab12e6806f


#### Start the Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [13]:
#create the server
server = create_server(server_name, 
                       reservation_id=compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [15]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

Floating IP: 192.5.87.221


## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit


In [14]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

chameleon_vlan  3291 , stitchport_url  http://geni-orca.renci.org/owl/ion.rdf#AL2S/iMind/Cisco/6509/TengigabitEthernet/1/1/ethernet , stitchport_vlan  1065 , geni_pem  /home/pruth/work/geni-pruth.pem
Opening certificate /home/pruth/work/geni-pruth.pem and key /home/pruth/work/geni-pruth.pem


log4j:WARN No appenders could be found for logger (org.renci.ahab.libndl.LIBNDL).
log4j:WARN Please initialize the log4j system properly.


#### Check the Status of the Circuit

In [16]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

chameleon_vlan  3291 , stitchport_url  http://geni-orca.renci.org/owl/ion.rdf#AL2S/iMind/Cisco/6509/TengigabitEthernet/1/1/ethernet , stitchport_vlan  1065 , geni_pem  /home/pruth/work/geni-pruth.pem
Opening certificate /home/pruth/work/geni-pruth.pem and key /home/pruth/work/geni-pruth.pem
statusSlice: uc-3291-customUrl-1065
Resource=sp2 State=Active
Resource=sp2 State=Active
Resource=sp2-sp1-net State=Active
state: STABLE


log4j:WARN No appenders could be found for logger (org.apache.commons.httpclient.HttpClient).
log4j:WARN Please initialize the log4j system properly.


## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [17]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

chameleon_vlan  3291 , stitchport_url  http://geni-orca.renci.org/owl/ion.rdf#AL2S/iMind/Cisco/6509/TengigabitEthernet/1/1/ethernet , stitchport_vlan  1065 , geni_pem  /home/pruth/work/geni-pruth.pem
Opening certificate /home/pruth/work/geni-pruth.pem and key /home/pruth/work/geni-pruth.pem
deleteSlice: uc-3291-customUrl-1065

log4j:WARN No appenders could be found for logger (org.apache.commons.httpclient.HttpClient).
log4j:WARN Please initialize the log4j system properly.


### Delete Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [18]:
delete_server_by_name(server_name)

()

#### De-configure Network
TODO: break up into steps

In [21]:
detach_router_by_name(router_name=router_name, subnet_name=subnet_name)

{'network_id': 'b6657377-18c2-4b7c-98c9-89dda0ec16b0',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '7c6b4fe8-60f8-414a-9010-a10504461049',
 'subnet_ids': ['7c6b4fe8-60f8-414a-9010-a10504461049'],
 'port_id': '38c74041-9652-479d-a6c0-d8c4547c8b7e',
 'id': '6f706200-c41b-48c7-8e46-b2647641c641'}

In [22]:
delete_router_by_name(router_name)

()

In [23]:
delete_subnet_by_name(subnet_name)

In [24]:
delete_network_by_name(network_name)

#### Release Leases

In [25]:
delete_lease_by_name(lease_name_network)
delete_lease_by_name(lease_name_servers)
delete_lease_by_name(lease_name_stitch)

Deleted lease pruthLeaseNet with id af1c661e-25e3-4e0e-bcb9-66b3730d4041
Deleted lease pruthLeaseServers with id c774245b-6ab1-4563-ad8e-e1efe9cbfb6c
Deleted lease pruthLeaseStitch with id 3a089e4b-91fe-43ba-b7e0-77154fa93e91
